In [1]:
using NeXLSpectrum
using NeXLParticle
using NeXLMatrixCorrection
using DataFrames
using Images
using FileIO
using Gadfly
using LinearAlgebra

┌ Info: Loading Gadfly support into NeXLSpectrum.
└ @ NeXLSpectrum C:\Users\nritchie\.julia\dev\NeXLSpectrum\src\gadflysupport.jl:564


In [2]:
path = @__DIR__

lt = 0.70*4.0*18.0*3600.0/(1024*1024) # 18.0 hours on 4 detectors
lt *= 1.133 # To bring the mean analytical total to ~1 
hs = NeXLSpectrum.compress(HyperSpectrum(
    LinearEnergyScale(0.0,10.0),
    Dict{Symbol,Any}(:ProbeCurrent => 1.0, :LiveTime => lt, :BeamEnergy => 20.0e3, :Name => splitdir(path)[2], :TakeOffAngle => deg2rad(35.0)),
    readrplraw(joinpath(path,"map[15]"))
));

In [3]:
refpath = joinpath(path, "Standards")

refs = references( [
    reference(n"Ag", joinpath(refpath, "Ag std.msa") ),
    reference(n"Al", joinpath(refpath, "Al std.msa") ),
    reference(n"C", joinpath(refpath, "C std.msa") ),
    reference(n"Ca", joinpath(refpath, "CaF2 std.msa") ),
    reference(n"Ce", joinpath(refpath, "CeO2 std.msa") ),
    reference(n"Cr", joinpath(refpath, "Cr std.msa") ),
    reference(n"Cu", joinpath(refpath, "Cu std.msa") ),
    reference(n"Fe", joinpath(refpath, "Fe std.msa") ),
    reference(n"S", joinpath(refpath, "FeS2 std.msa") ),
    reference(n"P", joinpath(refpath, "GaP std.msa") ),
    reference(n"K", joinpath(refpath, "KBr std.msa") ),
    reference(n"Mg", joinpath(refpath, "Mg std.msa") ),
    reference(n"O", joinpath(refpath, "MgO std.msa") ),
    reference(n"Mn", joinpath(refpath, "Mn std.msa") ),
    reference(n"Na", joinpath(refpath, "NaCl std.msa") ),
    reference(n"Ni", joinpath(refpath, "Ni std.msa") ),
    reference(n"Si", joinpath(refpath, "Si std.msa") ),
    reference(n"Ti", joinpath(refpath, "Ti std.msa") ),
    reference(n"Zn", joinpath(refpath, "Zn std.msa") ) ], 132.0
)
resf = fit_spectrum(hs[506:517,506:517], refs, mode = :Fast)

┌ Info: A material containing Ce and O is not a suitable reference for "Ce M5-N3 + 18 others" due to a peak interference.
└ @ NeXLSpectrum C:\Users\nritchie\.julia\dev\NeXLSpectrum\src\filter.jl:399


32-element Vector{KRatios}:
 k[Ag, Ag L3-M5 + 23 others] = Float32[ (12, 12) ]
 k[Al, Al K-L3 + 1 other] = Float32[ (12, 12) ]
 k[C, C K-L2] = Float32[ (12, 12) ]
 k[CaF2, Ca K-L3 + 3 others] = Float32[ (12, 12) ]
 k[CeO2, Ce L3-M5 + 30 others] = Float32[ (12, 12) ]
 k[Cr, Cr L3-M1 + 11 others] = Float32[ (12, 12) ]
 k[Cr, Cr K-L3 + 1 other] = Float32[ (12, 12) ]
 k[Cr, Cr K-M3 + 3 others] = Float32[ (12, 12) ]
 k[Cu, Cu L3-M5 + 11 others] = Float32[ (12, 12) ]
 k[Cu, Cu K-L3 + 1 other] = Float32[ (12, 12) ]
 k[Cu, Cu K-M3 + 3 others] = Float32[ (12, 12) ]
 k[Fe, Fe L3-M5 + 11 others] = Float32[ (12, 12) ]
 k[Fe, Fe K-L3 + 1 other] = Float32[ (12, 12) ]
 ⋮
 k[Mn, Mn K-L3 + 1 other] = Float32[ (12, 12) ]
 k[Mn, Mn K-M3 + 3 others] = Float32[ (12, 12) ]
 k[NaCl, Na K-L3 + 1 other] = Float32[ (12, 12) ]
 k[Ni, Ni L3-M5 + 11 others] = Float32[ (12, 12) ]
 k[Ni, Ni K-L3 + 1 other] = Float32[ (12, 12) ]
 k[Ni, Ni K-M3 + 3 others] = Float32[ (12, 12) ]
 k[Si, Si K-L3 + 2 others] = Float32[ (1

In [5]:
res = quantify(resf, mapfunc=map, resultfunc=iter->iter)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:35


12×12 Matrix{IterationResult}:
 Converged to CartesianIndex(1, 1)[S=0.0189,Ca=0.0202,Cu=0.0115,Ce=0.0037,Mn=0.3738,Mg=0.0137,Cr=0.0000,Ni=0.0204,O=0.3624,Zn=0.0000,K=0.0136,Ti=0.0011,Ag=0.0000,C=0.0813,Al=0.0065,Na=0.0344,P=0.0000,Fe=0.0117,Si=0.0211] in 10 steps
   …  Converged to CartesianIndex(1, 12)[S=0.0051,Ca=0.0141,Cu=0.0230,Ce=0.0003,Mn=0.3374,Mg=0.0155,Cr=0.0015,Ni=0.0133,O=0.3815,Zn=0.0043,K=0.0133,Ti=0.0000,Ag=0.0000,C=0.0947,Al=0.0125,Na=0.0223,P=0.0020,Fe=0.0174,Si=0.0235] in 11 steps

 Converged to CartesianIndex(2, 1)[S=0.0243,Ca=0.0149,Cu=0.0181,Ce=0.0028,Mn=0.3622,Mg=0.0161,Cr=0.0000,Ni=0.0217,O=0.3582,Zn=0.0000,K=0.0093,Ti=0.0003,Ag=0.0037,C=0.0745,Al=0.0088,Na=0.0362,P=0.0036,Fe=0.0046,Si=0.0080] in 10 steps
      Converged to CartesianIndex(2, 12)[S=0.0051,Ca=0.0162,Cu=0.0216,Ce=0.0000,Mn=0.3824,Mg=0.0217,Cr=0.0005,Ni=0.0157,O=0.3738,Zn=0.0000,K=0.0086,Ti=0.0025,Ag=0.0000,C=0.0616,Al=0.0069,Na=0.0064,P=0.0029,Fe=0.0121,Si=0.0178] in 11 steps

 Converged to Cartesian

In [6]:
(iter->analyticaltotal(iter.comp)).(res)

12×12 Matrix{Float64}:
 0.994492  1.02881   1.01038   0.977843  …  1.00812   0.979514  0.981876
 0.967246  1.04178   0.991609  1.11056      1.01747   1.03715   0.955908
 0.8951    0.929316  0.898858  0.971757     0.982542  1.04772   0.991652
 0.931799  1.07052   0.968076  1.03246      0.95425   1.0865    1.04121
 0.973928  0.742254  1.01801   1.03443      0.99034   0.981416  0.945819
 0.870801  0.826696  1.01262   1.04463   …  0.897768  0.999641  1.05605
 1.07275   1.01556   1.0523    1.02082      1.02401   1.08755   1.06859
 1.05797   1.03152   1.0084    0.944211     0.964677  0.975809  1.06222
 0.921165  0.981321  0.84371   0.948034     1.10779   1.03179   0.926674
 0.959689  0.943958  0.903794  0.914214     1.00647   1.0013    0.820206
 1.04328   1.06286   1.02874   1.02604   …  1.00069   1.1191    0.815053
 0.972602  0.970044  1.03661   1.07658      0.942668  1.00269   0.996727

In [7]:
using Profile
res = @profile quantify(resf, mapfunc=progress_map, resultfunc=iter->iter.comp)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:38


12×12 Matrix{Material{UncertainValue, Float64}}:
 CartesianIndex(1, 1)[S=0.0189,Ca=0.0202,Cu=0.0115,Ce=0.0037,Mn=0.3738,Mg=0.0137,Cr=0.0000,Ni=0.0204,O=0.3624,Zn=0.0000,K=0.0136,Ti=0.0011,Ag=0.0000,C=0.0813,Al=0.0065,Na=0.0344,P=0.0000,Fe=0.0117,Si=0.0211]   …  CartesianIndex(1, 12)[S=0.0051,Ca=0.0141,Cu=0.0230,Ce=0.0003,Mn=0.3374,Mg=0.0155,Cr=0.0015,Ni=0.0133,O=0.3815,Zn=0.0043,K=0.0133,Ti=0.0000,Ag=0.0000,C=0.0947,Al=0.0125,Na=0.0223,P=0.0020,Fe=0.0174,Si=0.0235]
 CartesianIndex(2, 1)[S=0.0243,Ca=0.0149,Cu=0.0181,Ce=0.0028,Mn=0.3622,Mg=0.0161,Cr=0.0000,Ni=0.0217,O=0.3582,Zn=0.0000,K=0.0093,Ti=0.0003,Ag=0.0037,C=0.0745,Al=0.0088,Na=0.0362,P=0.0036,Fe=0.0046,Si=0.0080]      CartesianIndex(2, 12)[S=0.0051,Ca=0.0162,Cu=0.0216,Ce=0.0000,Mn=0.3824,Mg=0.0217,Cr=0.0005,Ni=0.0157,O=0.3738,Zn=0.0000,K=0.0086,Ti=0.0025,Ag=0.0000,C=0.0616,Al=0.0069,Na=0.0064,P=0.0029,Fe=0.0121,Si=0.0178]
 CartesianIndex(3, 1)[S=0.0247,Ca=0.0204,Cu=0.0094,Ce=0.0008,Mn=0.3190,Mg=0.0052,Cr=0.0009,Ni=0.0109,O=0.3043

In [8]:
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
    ╎48755 @Base\task.jl:406; (::IJulia.var"#15#18")()
    ╎ 48755 ...ia\src\eventloop.jl:8; eventloop(socket::ZMQ.Socket)
    ╎  48755 @Base\essentials.jl:706; invokelatest
    ╎   48755 @Base\essentials.jl:708; #invokelatest#2
    ╎    48755 ...execute_request.jl:67; execute_request(socket::ZMQ....
    ╎     48755 ...oftGlobalScope.jl:65; softscope_include_string(m::...
    ╎    ╎ 48755 @Base\loading.jl:1094; include_string(mapexpr::ty...
  43╎    ╎  48755 @Base\boot.jl:360; eval
    ╎    ╎   86    ...ler\typeinfer.jl:921; typeinf_ext_toplevel(mi::...
    ╎    ╎    86    ...er\typeinfer.jl:925; typeinf_ext_toplevel(inte...
    ╎    ╎     86    ...er\typeinfer.jl:892; typeinf_ext(interp::Core...
    ╎    ╎    ╎ 86    ...er\typeinfer.jl:209; typeinf(interp::Core.Com...
    ╎    ╎    ╎  85    ...r\typeinfer.jl:214; _typeinf(interp::Core.C...
    ╎    ╎    ╎   85    ...rpretation.jl:1520; typeinf_nocycle(interp...
    ╎    ╎    ╎ 

Excessive output truncated after 524308 bytes.